In [16]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [17]:
prices = pd.read_csv('prices.csv')
wind = pd.read_csv('wind_power_cleaned.csv')
grid_excess = pd.read_csv('grid_excess.csv')

In [18]:
prices['date'] = pd.to_datetime(prices['date'])
wind['date'] = pd.to_datetime(wind['date'])

In [19]:
data = pd.merge(prices, wind, on=['date', 'hour'], how='outer')

#select every 18th day and save that to a new dataframe
data['date'] = pd.to_datetime(data['date'])
data

,date,hour,price,wind power
0,2023-01-01,0,2.01,114.776290
1,2023-01-01,1,1.38,117.265806
2,2023-01-01,2,0.09,117.016935
3,2023-01-01,3,0.08,113.002903
4,2023-01-01,4,0.05,108.146613
...,...,...,...,...
8756,2023-12-31,20,34.89,83.135484
8757,2023-12-31,21,29.60,72.667742
8758,2023-12-31,22,28.67,69.600000
8759,2023-12-31,23,26.87,72.667742


In [20]:
#fill nan values with the most recent value
data['wind power'] = data['wind power'].fillna(method='ffill')

C:\Users\janle\AppData\Local\Temp\ipykernel_14628\3486741652.py:2: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data['wind power'] = data['wind power'].fillna(method='ffill')


In [21]:
#get a list of every 18th day in 2023
selected_dates = pd.date_range(start='2023-01-01', end='2023-12-31', freq='19D')

In [22]:
#select every 18th day
scenarios_price = {}
scenarios_wind = {}
i=1
for date in selected_dates:
    scenarios_price[i] = prices[prices['date'] == date].reset_index(drop=True).drop(columns=['date', 'hour'])
    scenarios_wind[i] = wind[wind['date'] == date].reset_index(drop=True).drop(columns=['date', 'hour'])
    i+=1

In [23]:
#make 20 times 20 times 2 scenarios
scenarios = {}
i=1
for price in scenarios_price:
    for wind in scenarios_wind:
        for grid in grid_excess:
            df = pd.merge(scenarios_price[price], scenarios_wind[wind], left_index=True, right_index=True)
            #add column with grid excess
            df['grid_excess'] = grid_excess[grid]
            scenarios[i] = df
            i+=1

In [24]:
#pick 200 random scenarios and save them in one dataframe
import random
random.seed(42)
random_scenarios = random.sample(range(1, 1201), 250)

scenarios_200 = pd.DataFrame()
counter = 1
for i in random_scenarios:
    #add df columnwise to the dataframe without concat
    #scenarios[i].columns = [str(col) + '_'+str(counter) for col in scenarios[i].columns]
    scenarios_200 = pd.concat([scenarios_200, scenarios[i]] , axis=1)
    counter+=1

In [25]:
power = scenarios_200['wind power']
power

,wind power,wind power,wind power,wind power,wind power,wind power,wind power,wind power,wind power,wind power,...,wind power,wind power,wind power,wind power,wind power,wind power,wind power,wind power,wind power,wind power
0,151.992419,22.337242,21.359581,21.359581,66.631290,102.642258,41.843387,66.631290,15.778419,95.045161,...,31.444968,31.444968,114.776290,102.642258,64.397419,95.045161,95.045161,102.642258,21.359581,102.642258
1,143.459194,18.609871,17.627581,17.627581,62.685484,105.772419,37.184839,62.685484,14.820387,95.083226,...,34.361290,34.361290,117.265806,105.772419,65.745806,95.083226,95.083226,105.772419,17.627581,105.772419
2,142.586774,15.949935,15.520210,15.520210,56.905645,108.439516,35.290484,56.905645,16.245968,102.244516,...,34.861129,34.861129,117.016935,108.439516,66.184355,102.244516,102.244516,108.439516,15.520210,108.439516
3,146.618387,13.469581,15.687871,15.687871,45.898064,91.405968,26.789113,45.898064,21.937903,103.875484,...,42.039516,42.039516,113.002903,91.405968,68.079032,103.875484,103.875484,91.405968,15.687871,91.405968
4,150.135968,9.242903,13.684113,13.684113,39.361290,75.780645,13.576855,39.361290,20.196032,100.089839,...,46.991290,46.991290,108.146613,75.780645,67.271613,100.089839,100.089839,75.780645,13.684113,75.780645
5,151.101613,5.908274,6.538097,6.538097,45.957903,61.308065,10.595290,45.957903,19.268613,100.234193,...,46.091290,46.091290,109.434839,61.308065,68.490161,100.234193,100.234193,61.308065,6.538097,61.308065
6,145.926935,4.519484,3.912145,3.912145,48.470323,48.278548,10.433968,48.470323,17.511322,98.578427,...,43.650323,43.650323,104.349194,48.278548,73.088387,98.578427,98.578427,48.278548,3.912145,48.278548
7,143.261129,2.733911,6.318161,6.318161,47.922742,35.350903,13.341064,47.922742,17.589597,101.974516,...,47.518387,47.518387,103.625806,35.350903,70.643710,101.974516,101.974516,35.350903,6.318161,35.350903
8,142.278710,1.103897,11.046871,11.046871,49.915968,25.439984,19.462226,49.915968,18.876742,106.932581,...,52.517903,52.517903,103.569839,25.439984,66.540806,106.932581,106.932581,25.439984,11.046871,25.439984
9,142.215323,0.977411,12.637242,12.637242,55.955161,23.913387,25.728968,55.955161,18.812710,111.288710,...,64.439032,64.439032,104.884516,23.913387,68.425484,111.288710,111.288710,23.913387,12.637242,23.913387


In [26]:
#save as csv
scenarios_200.to_csv('scenarios_200.csv', index=False)